## Preliminaries

This iPython notebook is written to play around with a small pure integer problem. 
First random data is generated for a problem in standard form.
After this, the LP relaxation is solved (using a call to CPLEX)
After this, GMI cuts are added and the new LP relaxation is solved
After this X cuts and GX cuts are added and the new LP relaxations are solved
Finally the IP is solved
Then a summary of minimization objective value obtained in each of these is printed.

### Library loading

In [1]:
%load_ext autoreload
%autoreload

import numpy as np
import scipy as sp
import copy


from Cuts import *
from ReprVars import *
from CPLEXInterface import *

### Problem size definition

In [2]:
Nvar = 5
Ncons = 2
Num_IP = 1
Num_X_inst = 5


### Data generation

In [8]:
# Data generation
A = np.random.randint(-10, 10, (Ncons, Nvar))
b = np.random.randint(0, 100, (Ncons, 1))
f = np.random.randint(-100, 100, (Nvar, 1))
cont_var = np.zeros((Nvar, 1))
int_var = 1-cont_var;

# print(A, b, f, cont_var)

In [9]:
# A = np.array([
#     [-10,-5,5,6,1],
#     [6,7,2,-5,0]
# ])
# b = np.array([
#     [86],
#     [95]
# ])
# f = np.array([
#     [30],
#     [17],
#     [62],
#     [46],
#     [42]
# ])
print(A, b, f)

[[ 5 -6 -7  4 -6]
 [-5  7 -3 -4  9]] [[66]
 [43]] [[ -2]
 [ 28]
 [ 43]
 [ 95]
 [-17]]


### Solving LP relaxation from CPLEX and retrieving data from CPLEX

In [10]:
M = MIP(form = 1, data = {
    'f':f,
    'Aeq': A,
    'beq': b,
    'cont': cont_var
})
#print(M.f, M.Aeq, M.beq, M.cont)
M_cplex = Py2Cplex(M)
LPSolution = getfromCPLEX(M_cplex)
print()
print()
[print(p,':', q) for p,q in LPSolution.items()]




Tableaux : [[ 0.          0.33333333 -3.33333333  0.          1.        ]
 [ 1.         -0.8        -5.4         0.8         0.        ]]
NonBasic : [1 2 3]
Solution : [[ 56.8       ]
 [  0.        ]
 [  0.        ]
 [  0.        ]
 [ 36.33333333]]
Basic : [0 4]
Objective : -731.2666666666668


[None, None, None, None, None]

In [12]:
print('Basis:',M_cplex.solution.basis.get_basis())

Basis: ([1, 0, 0, 0, 1], [0, 0])


In [44]:
M_cplex.write("smallLP.mps")

Default column names x1, x2 ... being created.
Default row    names c1, c2 ... being created.


## GMI cuts

### Cut generation

In [32]:
(A_GMI, b_GMI) = GMI(
    LPSolution["Tableaux"][:, LPSolution["NonBasic"]], 
    LPSolution["Solution"], 
    int_var[LPSolution["Basic"]].astype(int), 
    cont_var[LPSolution["NonBasic"]].astype(int)
)
print(int_var[LPSolution["Basic"]].astype(int))
print(A_GMI, b_GMI)

[[1]
 [1]]
[[ 0.42105263  0.31578947  0.89473684]
 [ 0.15789474  0.63157895  0.28947368]] [[ 1.]
 [ 1.]]


### Adding cuts and solving new LP

In [33]:
GMI_M = addCuts(M, LPSolution["NonBasic"], A_GMI, b_GMI)
GMI_cplex = Py2Cplex(GMI_M)
GMIans = getfromCPLEX(GMI_cplex)
print()
[print(p,':', q) for p,q in GMIans.items()]

CPXPARAM_Preprocessing_Fill                      0
CPXPARAM_Preprocessing_Aggregator                0
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Preprocessing_NumPass                   0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_Reduce                    0
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_LPMethod                                1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 0 times.
Presolve time = 0.00 sec. (0.00 ticks)

Tableaux : [[ 1.          2.9122807   0.          0.          0.         -2.66666667
   0.33333333]
 [ 0.          0.04093567  1.          0.          0.          0.61111111
  -1.88888889]
 [ 0.          0.87719298  0.          0.          1.         -1.83333333
   0.66666667]
 [ 0.          0.45614035  0.          1.          0.         -1.33333333
   0.66666667]]
NonBasic : [1 5 6]
Objective : 382.9444444444499
B

[None, None, None, None, None]

In [34]:
print("Before GMI", M.f.T.dot(LPSolution["Solution"]))
print("After GMI", GMI_M.f.T.dot(GMIans["Solution"]))

print(M.Aeq, M.beq)
print(GMI_M.Aeq, GMI_M.beq)

Before GMI [[ 278.]]
After GMI [[ 382.94444444]]
[[ -2  -2   0  -7   8]
 [  5   9  -3 -10  -1]] [[89]
 [72]]
[[ -2.          -2.           0.          -7.           8.           0.
    0.        ]
 [  5.           9.          -3.         -10.          -1.           0.
    0.        ]
 [  0.          -0.42105263  -0.31578947  -0.89473684   0.           1.
    0.        ]
 [  0.          -0.15789474  -0.63157895  -0.28947368   0.           0.
    1.        ]] [[ 89.]
 [ 72.]
 [ -1.]
 [ -1.]]


## X cuts and GX cuts

### Generating RowMat and muMat

In [35]:
x_B = LPSolution["Solution"][LPSolution["Basic"]]
print(x_B)
ans = Rows4Xcut(x_B, 2, 5, int_var[LPSolution["Basic"]], 1)
#print(ans["muMat"], ans["fMat"], ans["RowMat"])

[[ 17.5]
 [ 15.5]]


### Generating X cuts

In [36]:
(A_X, b_X) = XLift(
    LPSolution["Tableaux"][:, LPSolution["NonBasic"]], 
    LPSolution["Solution"][LPSolution["Basic"]],
    ans["RowMat"],
    ans["muMat"],
    cont_var[LPSolution["NonBasic"]].astype(int)
)
print(A_X, b_X)

cut 1 generated
cut 2 generated
cut 3 generated
cut 4 generated
cut 5 generated
[[ 0.37353363  0.50586547  0.75217984]
 [ 0.37813459  0.48746163  0.76598272]
 [ 0.37622988  0.49508048  0.76026859]
 [ 0.36947133  0.52211467  0.73999295]
 [ 0.38756738  0.44973046  0.7942811 ]] [[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]


### Adding the cuts and solving LP

In [37]:
X_M = addCuts(M, LPSolution["NonBasic"], A_X, b_X)
X_cplex = Py2Cplex(X_M)
Xans = getfromCPLEX(X_cplex)
[print(p,' : ',q) for p,q in Xans.items()]

CPXPARAM_Preprocessing_Fill                      0
CPXPARAM_Preprocessing_Aggregator                0
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Preprocessing_NumPass                   0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_Reduce                    0
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_LPMethod                                1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 0 times.
Presolve time = 0.00 sec. (0.00 ticks)
Tableaux  :  [[ 0.          0.          0.          0.          0.          1.          0.
   0.         -0.77551465 -0.22448535]
 [ 0.         -0.          0.          0.          0.          0.          1.
   0.         -0.52126249 -0.47873751]
 [ 0.          0.          0.          0.          0.          0.          0.
   1.         -0.62651814 -0.37348186]
 [ 1.          2.90514076  0.          0.          0.  

[None, None, None, None, None]

### Generating GX cuts

In [38]:
(A_GX, b_GX) = GXLift(
    LPSolution["Tableaux"][:, LPSolution["NonBasic"]], 
    LPSolution["Solution"][LPSolution["Basic"]],
    ans["RowMat"],
    ans["muMat"],
    ans["fMat"],
    cont_var[LPSolution["NonBasic"]].astype(int)
)
print(A_GX, b_GX)
print(LPSolution["Tableaux"][:, LPSolution["NonBasic"]])

cut 0generated
cut 1generated
cut 2generated
cut 3generated
cut 4generated
[[ 0.3363365   0.320737    0.88016368]
 [ 0.4193184   0.69318583  0.76151048]
 [ 0.34371062  0.35641411  0.814398  ]
 [ 0.32384827  0.42409179  0.62247477]
 [ 0.39775184  0.47391461  0.84202846]] [[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]
[[ 0.21052632 -0.15789474 -1.44736842]
 [ 1.84210526 -0.63157895 -2.28947368]]


### Adding the cuts and solving LP

In [39]:
GX_M = addCuts(M, LPSolution["NonBasic"], A_GX, b_GX)
GX_cplex = Py2Cplex(GX_M)
GXans = getfromCPLEX(GX_cplex)
[print(p,' : ',q) for p,q in GXans.items()]

CPXPARAM_Preprocessing_Fill                      0
CPXPARAM_Preprocessing_Aggregator                0
CPXPARAM_Preprocessing_Presolve                  0
CPXPARAM_Preprocessing_NumPass                   0
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Preprocessing_Reduce                    0
CPXPARAM_Preprocessing_Linear                    0
CPXPARAM_LPMethod                                1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 0 times.
Presolve time = 0.00 sec. (0.00 ticks)
Tableaux  :  [[  0.00000000e+00   4.35887104e-01   1.00000000e+00   0.00000000e+00
    0.00000000e+00   3.58528139e+00   0.00000000e+00   0.00000000e+00
   -5.06949784e+00   0.00000000e+00]
 [  0.00000000e+00   5.28697177e-02   0.00000000e+00   0.00000000e+00
    0.00000000e+00   6.25162188e-01   1.00000000e+00   0.00000000e+00
   -2.10732323e+00   0.00000000e+00]
 [  0.00000000e+00  -6.50771772e-03   0.00000000e+00

[None, None, None, None, None]

In [40]:
IP_M = Py2Cplex(M)

In [41]:
for i in np.arange(Nvar):
    IP_M.variables.set_types(int(i), IP_M.variables.type.integer)
    
IP_M.set_results_stream(None)
IP_M.solve()
print('Status : ', IP_M.solution.status[IP_M.solution.get_status()])
IP_M.solution.get_status()
#print('IP Objective: ',IP_M.solution.get_objective_value())

Status :  MIP_optimal


101

## Summary

In [42]:
Summary = {"1. LP obj": LPSolution["Objective"],
          "2. With GMI Cuts": GMIans["Objective"],
          "3. With X-Cuts": Xans["Objective"],
          "4. With GX-Cuts": GXans["Objective"]          
          }
if IP_M.solution.get_status() == 103:
    Summary["5. IP obj"] = np.inf
else:
    Summary["5. IP obj"] = IP_M.solution.get_objective_value()
    
[print(p," : ",q) for p,q in sorted(Summary.items())]
print()

1. LP obj  :  278.0
2. With GMI Cuts  :  382.9444444444499
3. With X-Cuts  :  372.1279069767486
4. With GX-Cuts  :  386.68519761609184
5. IP obj  :  543.0

